In [1]:
! pip install pyspark==3.3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 5.2 MB/s eta 0:00:00a 0:00:01
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845494 sha256=b0c61c18711c016118328be9797456a71e35e4a0e19102a7f401efb568c59006
  Stored in directory: /home/airflow/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.1
    Uninstalling pyspark-3.5.1:
      Successfully uninstalled pyspark-3.5.1

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession\
    .builder\
    .appName("twitter_transformation")\
    .getOrCreate()

/home/airflow/.local/lib/python3.8/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/25 20:31:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/25 20:31:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
df = spark.read.json("/home/airflow/workspaces/airflow-spark/data/bronze/twitter_datascience")

In [5]:
df.show()

+--------------------+--------------------+------------------+------------+
|                data|            includes|              meta|extract_date|
+--------------------+--------------------+------------------+------------+
|[{67, 1, 2024-03-...|{[{2024-03-25T15:...|{1234567890abcdef}|  2024-03-25|
|[{60, 23, 2024-03...|{[{2024-03-24T13:...|              null|  2024-03-25|
+--------------------+--------------------+------------------+------------+



In [6]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- c

In [7]:
from pyspark.sql import functions as f

In [8]:
df.select(f.explode("data")).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- author_id: string (nullable = true)
 |    |-- conversation_id: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- id: string (nullable = true)
 |    |-- in_reply_to_user_id: string (nullable = true)
 |    |-- lang: string (nullable = true)
 |    |-- public_metrics: struct (nullable = true)
 |    |    |-- like_count: long (nullable = true)
 |    |    |-- quote_count: long (nullable = true)
 |    |    |-- reply_count: long (nullable = true)
 |    |    |-- retweet_count: long (nullable = true)
 |    |-- text: string (nullable = true)



In [9]:
df.select(f.explode("data")).show()

+--------------------+
|                 col|
+--------------------+
|{67, 1, 2024-03-2...|
|{51, 97, 2024-03-...|
|{21, 83, 2024-03-...|
|{86, 84, 2024-03-...|
|{54, 79, 2024-03-...|
|{5, 70, 2024-03-2...|
|{46, 44, 2024-03-...|
|{47, 24, 2024-03-...|
|{60, 67, 2024-03-...|
|{65, 27, 2024-03-...|
|{60, 23, 2024-03-...|
|{14, 5, 2024-03-2...|
|{72, 85, 2024-03-...|
|{99, 90, 2024-03-...|
|{83, 55, 2024-03-...|
|{20, 21, 2024-03-...|
|{89, 70, 2024-03-...|
|{61, 11, 2024-03-...|
|{43, 74, 2024-03-...|
|{23, 35, 2024-03-...|
+--------------------+



In [10]:
df.select(f.explode("data").alias("tweets"))\
  .select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text").printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)



In [11]:
tweet_df = df.select(f.explode("data").alias("tweets"))\
  .select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text")

In [12]:
tweet_df.show(5)

+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|author_id|conversation_id|          created_at| id|like_count|quote_count|reply_count|retweet_count|                text|
+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|       67|              1|2024-03-25T06:05:...| 84|        65|         95|         22|           24|Outro tweet fictí...|
|       51|             97|2024-03-25T07:23:...| 36|        32|         73|         20|            5|Tweet fictício ge...|
|       21|             83|2024-03-24T16:35:...| 94|        97|         75|         64|           68|Tweet fictício ge...|
|       86|             84|2024-03-24T23:48:...|  2|        57|         83|          2|           58|Outro tweet fictí...|
|       54|             79|2024-03-24T05:39:...| 41|        60|         50|         79|           96|Outro tweet fictí...|
+---------+-----

In [13]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- c

In [14]:
df.select(f.explode("includes.users").alias("users")).select("users.*").printSchema()

root
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- username: string (nullable = true)



In [15]:
user_df = df.select(f.explode("includes.users").alias("users")).select("users.*")

In [16]:
user_df.show(5)

+--------------------+---+------+--------+
|          created_at| id|  name|username|
+--------------------+---+------+--------+
|2024-03-25T15:34:...| 70|User 1|   user1|
|2024-03-25T19:15:...| 65|User 2|   user2|
|2024-03-24T08:55:...| 83|User 3|   user3|
|2024-03-24T07:02:...|  7|User 4|   user4|
|2024-03-25T03:00:...| 79|User 5|   user5|
+--------------------+---+------+--------+
only showing top 5 rows



In [17]:
tweet_df.coalesce(1).write.mode("overwrite").json('output/tweet')
user_df.coalesce(1).write.mode("overwrite").json('output/user')